### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
import oci
from oci.data_science import DataScienceClient
from oci.data_science.models import CreateModelDetails, CreateProjectDetails, \
    CreateModelProvenanceDetails, Project, Model, UpdateModelDetails
import os
import logging

REGION = "us-ashburn-1"
logger = logging.getLogger('upload_model_artifact')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

def fetch_config() -> dict:
    """
    Gets the oci config stored locally.

    Returns:
    config (dict): a config object
    """

    try:
        config = oci.config.from_file()
        config['region'] = REGION

        # We can see what our config file looks like by using 'print(config)'.
        return config
    except Exception as e:
        logger.error("Failed while getting oci config : %s", format(e))
        exit(1)
        
config = fetch_config()

In [ ]:
SERVICE_ENDPOINT = "https://datascience.us-phoenix-1.oci.oraclecloud.com/20190101"

def create_data_science_client(config: dict) -> DataScienceClient:
    """
    Creates a data science client using the config.

    Parameters:
    config (dict): the oci config

    Returns:
    data_science (DataScienceClient): a data science client object
    """

    logger.info("Setting up data science client and overriding service endpoint...")
    try:
        data_science = DataScienceClient(config, service_endpoint=SERVICE_ENDPOINT)
        logger.info("Successfully set up data science client!")
        return data_science
    except Exception as e:
        logger.error("Failed to set up data science client with error: %s", format(e))
        exit(1)
        
data_science_client = create_data_science_client(config)

In [ ]:
# set Compartment Id
COMPARTMENT_ID = os.environ['NB_SESSION_COMPARTMENT_OCID']
PROJECT_DESCRIPTION = "Test Project"
PROJECT_DISPLAY_NAME = "ModelStore-ArtifactTest"

data_science_models = oci.data_science.models

def create_project_details(data_science_models) -> CreateProjectDetails:
    """
    Creates the project details.

    Parameters:
    data_science_models (module): the data science models to import create project details model from

    Returns:
    create_project_details_object (CreateProjectDetails): a create project details model object
    """

    logger.info("Defining project details object...")
    # We need to create a project first. Get the create project details object.
    create_project_details_object = data_science_models.CreateProjectDetails()

    # Set the project compartment id.
    create_project_details_object.compartment_id = COMPARTMENT_ID

    # Set the project display name.
    create_project_details_object.display_name = PROJECT_DISPLAY_NAME

    # Set the project description
    create_project_details_object.description = PROJECT_DESCRIPTION
    # logger.info("Project details: %s", create_project_details_object)
    return create_project_details_object
create_project_details_object = create_project_details(data_science_models)

In [ ]:
def create_project(data_science_client: DataScienceClient, create_project_details_object: CreateProjectDetails) -> str:
    """
    Creates the project.

    Parameters:
    data_science_client (DataScienceClient): the data science client
    create_project_details_object (CreateProjectDetails): the create project details model object

    Returns:
    project_id (str): a project ocid
    """

    try:
        logger.info("Creating project...")
        # print(data_science_client.base_client.__dict__)
        project = data_science_client.create_project(create_project_details_object)
        # logger.info("Data returned: %s", project.data)
        # logger.info("Grabbing the project ocid...")
        project_id = json.loads(str(project.data))['id']
        # logger.info("Project ocid: %s", project_id)
        return project_id
    except Exception as e:
        logger.error("Failed to create project with error: %s", format(e))
        exit(1)

In [ ]:
def create_project_main():
    project_id = create_project(data_science_client, create_project_details_object)
    logger.info(project_id)
    return str(project_id)

project_id = create_project_main()

In [ ]:
def create_model_details(data_science_models, project_id: str):
    """
    Creates the model details object

    Parameters:
    data_science_models (module): the data science models to import create model details model from
    project_id (str): the project ocid

    Returns:
    create_model_details_object (CreateModelDetails): a create model details object
    """
    logger.info("Defining Model details object...")
    create_model_details_object = data_science_models.CreateModelDetails()
    create_model_details_object.compartment_id = COMPARTMENT_ID
    create_model_details_object.display_name = "MD-ModelArtifact-test"
    create_model_details_object.project_id = project_id
    return create_model_details_object

def create_model(data_science_client: DataScienceClient, create_model_details_object: CreateModelDetails):
    """
    Creates the model.

    Parameters:
    data_science_client (DataScienceClient): the data science client
    create_model_details_object (CreateModelDetails): a create model details object to use in creating models

    Returns:
    model_id (str): the model ocid generated.
    """
    try:
        logger.info("Creating model...")
        model = data_science_client.create_model(
            create_model_details_object)
        logger.info(f'model details: {model}')
        model_id = json.loads(str(model.data))['id']
        return model_id
    except Exception as e:
        logger.error("Failed to create model with error: %s", format(e))
        exit(1)

In [ ]:
def create_model_main(project_id: str):
    # def create_model_main():
    logger.info("creating model...")
    # creates the model and model artifact
    # Make sure to provide the correct path of the zip file in ARTIFACT_FILE_NAME
    create_model_details_object = create_model_details(data_science_models, project_id)
    model_id = create_model(data_science_client, create_model_details_object)
    # create_model_artifact(data_science_client, model_id)
    logger.info(model_id)
    return str(model_id)

model_id = create_model_main(create_project_main())